In [21]:
import featuretools as ft
import pandas as pd
ORIGINAL_TRAIN = pd.read_csv('train.csv',engine = 'pyarrow')

In [22]:
ORIGINAL_TRAIN.columns

Index(['stock_id', 'date_id', 'seconds_in_bucket', 'imbalance_size',
       'imbalance_buy_sell_flag', 'reference_price', 'matched_size',
       'far_price', 'near_price', 'bid_price', 'bid_size', 'ask_price',
       'ask_size', 'wap', 'target', 'time_id', 'row_id'],
      dtype='object')

截面算子生成

In [24]:
ORIGINAL_TRAIN.tail(10)

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
5237970,190,480,540,466320.15,-1,0.999295,3.929472e+07,0.998983,0.999139,0.999295,19725.86,0.999451,169105.20,0.999311,-0.470281,26454,480_540_190
5237971,191,480,540,36937785.44,1,0.995983,6.083261e+08,0.999086,0.997746,0.995913,84744.00,0.995983,48025.00,0.995958,0.799894,26454,480_540_191
5237972,192,480,540,67162.62,1,1.000762,1.179008e+07,1.000834,1.000762,1.000545,168412.86,1.000762,69841.50,1.000698,-1.729727,26454,480_540_192
5237973,193,480,540,6500948.04,1,1.001599,8.035753e+07,1.002908,1.002326,1.001454,413142.24,1.001599,363532.53,1.001531,-3.880262,26454,480_540_193
5237974,194,480,540,262271.52,-1,1.001504,3.922295e+06,1.000677,1.000677,1.001504,20829.20,1.001917,139637.49,1.001557,4.839897,26454,480_540_194
5237975,195,480,540,2440722.89,-1,1.000317,2.828036e+07,0.999734,0.999734,1.000317,32257.04,1.000434,319862.40,1.000328,2.310276,26454,480_540_195
5237976,196,480,540,349510.47,-1,1.000643,9.187699e+06,1.000129,1.000386,1.000643,205108.40,1.000900,93393.07,1.000819,-8.220077,26454,480_540_196
5237977,197,480,540,0.00,0,0.995789,1.272544e+07,0.995789,0.995789,0.995789,16790.66,0.995883,180038.32,0.995797,1.169443,26454,480_540_197
5237978,198,480,540,1000898.84,1,0.999210,9.477327e+07,0.999210,0.999210,0.998970,125631.72,0.999210,669893.00,0.999008,-1.540184,26454,480_540_198
5237979,199,480,540,1884285.71,-1,1.002129,2.407368e+07,1.000859,1.001494,1.002129,250081.44,1.002447,300167.56,1.002274,-6.530285,26454,480_540_199


In [9]:
#提取目标字段
target=ORIGINAL_TRAIN['target']
ORIGINAL_TRAIN.drop(['target'],axis=1,inplace=True)

In [10]:
tram=ft.list_primitives()
tram
transform_factor=tram.query("type=='transform'")
aggregation_factor=tram.query("type=='aggregation'")
agg_primitives_all=aggregation_factor.iloc[:,0].values
trans_primitives_all=transform_factor.iloc[:,0].values
print(agg_primitives_all)

['max_consecutive_negatives' 'entropy' 'percent_unique' 'all'
 'first_last_time_delta' 'max' 'num_unique' 'time_since_last_max'
 'count_outside_range' 'n_unique_days_of_month' 'time_since_first'
 'count_less_than' 'n_unique_months' 'is_monotonically_decreasing'
 'count_inside_nth_std' 'median_count' 'skew' 'max_consecutive_zeros'
 'n_most_common' 'time_since_last_min' 'count_above_mean' 'median'
 'max_consecutive_positives' 'percent_true' 'average_count_per_unique'
 'is_monotonically_increasing' 'num_true_since_last_false' 'num_peaks'
 'avg_time_between' 'mean' 'time_since_last_false'
 'n_unique_days_of_calendar_year' 'max_consecutive_false' 'variance' 'any'
 'has_no_duplicates' 'kurtosis' 'time_since_last' 'first'
 'num_consecutive_less_mean' 'sum' 'count_outside_nth_std' 'min'
 'n_most_common_frequency' 'std' 'count_inside_range' 'mode'
 'num_consecutive_greater_mean' 'count_greater_than' 'last'
 'num_false_since_last_true' 'count_below_mean' 'min_count'
 'max_consecutive_true' 'coun

In [19]:
import featuretools as ft

price_set = ORIGINAL_TRAIN
#运行太慢了，示例前100行
price_set= price_set.head(100)

def featuretools_generate_price(df):

    # 创建实体集
    es = ft.EntitySet(id="stocks")

    # 添加主实体 `stock`
    es = es.add_dataframe(dataframe_name="stock", dataframe=df[['stock_id', 'date_id', 'time_id']], index="stock_id")
    # 可以用date_id做主键，这样就是做同一天内不同的stock的算子指标
    #es = es.add_dataframe(dataframe_name="stock", dataframe=df[['stock_id', 'date_id', 'time_id']], index="date_id")
    # 添加子实体 `stock_trades`
    es = es.add_dataframe(dataframe_name="stock_trades", dataframe=df, index="row_id")

    # 定义关系
    es = es.add_relationship("stock", "stock_id", "stock_trades", "stock_id")

    # 执行 DFS
    feature_matrix, feature_names = ft.dfs(entityset=es,
                                       target_dataframe_name="stock",
                                       agg_primitives=['mean', 'sum', 'std', 'min', 'max', 'count', 'num_unique'],
                                       trans_primitives=['time_since_previous', 'cum_sum', 'percentile', 'divide_numeric', 'absolute'],
                                       max_depth=2)
    return feature_names, feature_matrix

# 生成特征
feature_names, feature_matrix = featuretools_generate_price(price_set)


C:\Users\yigaa\.conda\envs\MLUsed\Lib\site-packages\woodwork\type_sys\utils.py:33: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(
C:\Users\yigaa\.conda\envs\MLUsed\Lib\site-packages\woodwork\type_sys\utils.py:33: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(
C:\Users\yigaa\.conda\envs\MLUsed\Lib\site-packages\featuretools\synthesis\dfs.py:321: UnusedPrimitiveWarning: Some specified primitives were not used during DFS:
  trans_primitives: ['time_since_previous']
  agg_primitives: ['num_unique']
This may be caused by a using a value of max_depth that is too small, not setting interesting values, or it may indicate no compatible columns for the primitive were found in the data. If th

In [20]:
feature_matrix

,date_id,time_id,COUNT(stock_trades),MAX(stock_trades.ask_price),MAX(stock_trades.ask_size),MAX(stock_trades.bid_price),MAX(stock_trades.bid_size),MAX(stock_trades.date_id),MAX(stock_trades.imbalance_buy_sell_flag),MAX(stock_trades.imbalance_size),...,PERCENTILE(SUM(stock_trades.bid_price)),PERCENTILE(SUM(stock_trades.bid_size)),PERCENTILE(SUM(stock_trades.date_id)),PERCENTILE(SUM(stock_trades.imbalance_buy_sell_flag)),PERCENTILE(SUM(stock_trades.imbalance_size)),PERCENTILE(SUM(stock_trades.matched_size)),PERCENTILE(SUM(stock_trades.reference_price)),PERCENTILE(SUM(stock_trades.seconds_in_bucket)),PERCENTILE(SUM(stock_trades.time_id)),PERCENTILE(SUM(stock_trades.wap))
stock_id,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,1.000026,8493.03,0.999812,60651.50,0.0,1.0,3180602.69,...,0.560,0.90,0.505,0.795,0.650,0.82,0.340,0.505,0.505,0.505
1,0,0,1,1.000660,20605.09,0.999896,3233.04,0.0,-1.0,166603.91,...,0.720,0.19,0.505,0.205,0.240,0.12,0.430,0.505,0.505,0.505
2,0,0,1,1.000298,18995.00,0.999403,37956.00,0.0,-1.0,302879.87,...,0.250,0.78,0.505,0.205,0.280,0.17,0.210,0.505,0.505,0.505
3,0,0,1,1.000214,479032.40,0.999999,2324.90,0.0,-1.0,11917682.27,...,0.995,0.15,0.505,0.205,0.890,0.89,0.700,0.505,0.505,0.505
4,0,0,1,1.000016,434.10,0.999394,16485.54,0.0,-1.0,447549.96,...,0.230,0.59,0.505,0.205,0.310,0.87,0.170,0.505,0.505,0.505
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,0,0,1,1.001055,483973.71,0.999694,140039.77,0.0,1.0,23143.83,...,0.410,0.96,0.505,0.795,0.190,0.06,0.860,0.505,0.505,0.505
99,0,0,1,1.000097,28525.00,0.999851,43958.46,0.0,1.0,3365706.48,...,0.660,0.81,0.505,0.795,0.690,0.41,0.110,0.505,0.505,0.505
100,0,0,1,1.000209,6241.62,0.999839,4807.78,0.0,0.0,0.00,...,0.610,0.24,0.505,0.495,0.095,0.01,0.730,0.505,0.505,0.505


多线程并行计算：
OPENFE
parallel
tsfresh和featuretools内本身存在并行计算n_jobs


时序算子生成

In [29]:
import pandas as pd
from tsfresh import extract_features, select_features
from tsfresh.utilities.dataframe_functions import impute

price_set = ORIGINAL_TRAIN
# 使用前 1000 行数据
price_set = price_set.head(1000)
# 填补缺失值
price_set = price_set.fillna(0)

# 将数据转换为长格式
data_long = pd.melt(price_set, id_vars=['stock_id', 'seconds_in_bucket'], 
                    value_vars=['imbalance_size', 'reference_price', 'matched_size', 'far_price', 'near_price', 
                                'bid_price', 'bid_size', 'ask_price', 'ask_size', 'wap', 'target'],
                    var_name='variable', value_name='value')

# 提取特征
extracted_features = extract_features(data_long, 
                                      column_id='stock_id', 
                                      column_sort='seconds_in_bucket', 
                                      column_value='value',
                                      column_kind='variable')

# 创建以 stock_id 为索引的 target Series
target = price_set.groupby('stock_id')['target'].first()

# 确保 extracted_features 和 target 具有相同的索引
common_ids = extracted_features.index.intersection(target.index)
extracted_features = extracted_features.loc[common_ids]
target = target.loc[common_ids]

# 填补缺失值
impute(extracted_features)

# 进行特征选择
relevant_features = select_features(extracted_features, target)



Feature Extraction: 100%|██████████| 54/54 [00:34<00:00,  1.55it/s]
C:\Users\yigaa\.conda\envs\MLUsed\Lib\site-packages\tsfresh\utilities\dataframe_functions.py:198: RuntimeWarning: The columns ['ask_price__autocorrelation__lag_6' 'ask_price__autocorrelation__lag_7'
 'ask_price__autocorrelation__lag_8' ...
 'wap__permutation_entropy__dimension_7__tau_1'
 'wap__query_similarity_count__query_None__threshold_0.0'
 'wap__mean_n_absolute_max__number_of_maxima_7'] did not have any finite values. Filling with zeros.
  warnings.warn(


In [32]:
relevant_features.columns

Index(['target__cwt_coefficients__coeff_0__w_2__widths_(2, 5, 10, 20)',
       'target__linear_trend__attr_"intercept"',
       'wap__change_quantiles__f_agg_"mean"__isabs_False__qh_1.0__ql_0.0',
       'wap__mean_change', 'wap__c3__lag_2', 'wap__linear_trend__attr_"slope"',
       'wap__root_mean_square', 'wap__mean',
       'target__cwt_coefficients__coeff_0__w_5__widths_(2, 5, 10, 20)',
       'wap__time_reversal_asymmetry_statistic__lag_2',
       ...
       'ask_price__agg_linear_trend__attr_"slope"__chunk_len_5__f_agg_"min"',
       'bid_size__ratio_value_number_to_time_series_length',
       'bid_size__percentage_of_reoccurring_values_to_all_values',
       'bid_size__percentage_of_reoccurring_datapoints_to_all_datapoints',
       'wap__ratio_beyond_r_sigma__r_1', 'bid_price__autocorrelation__lag_2',
       'ask_size__index_mass_quantile__q_0.1', 'ask_price__has_duplicate_min',
       'ask_price__change_quantiles__f_agg_"var"__isabs_False__qh_1.0__ql_0.0',
       'bid_price__lin